In [ ]:
%pip install ddgs

/opt/homebrew/Cellar/python@3.13/3.13.4/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=37802) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [52]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool,OpenAIChatCompletionsModel
from ddgs import DDGS
from openai import AsyncOpenAI
from agents.model_settings import ModelSettings
from pydantic import BaseModel,Field
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from IPython.display import display, Markdown
from email.message import EmailMessage
import ssl
import smtplib

In [32]:
load_dotenv(override=True)

True

In [33]:
client = AsyncOpenAI(
    api_key=os.getenv("GEMINI_ACCESS_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

In [37]:
class SearchInput(BaseModel):
    query: str


@function_tool
def duckduckgo_web_search(input: SearchInput) -> str:
    """Search the web using DuckDuckGo and return text snippets."""
    with DDGS() as ddgs:
        results = ddgs.text(input.query, max_results=20)
        filtered = [r["body"] for r in results if "agent" in r["body"].lower()]
        return "\n".join(filtered) or "No relevant results found."

<h2>OpenAI Hosted Tools</h2>
OpenAI Agents SDK includes the following hosted tools:

The WebSearchTool lets an agent search the web.<br>
The FileSearchTool allows retrieving information from your OpenAI Vector Stores.<br>
The ComputerTool allows automating computer use tasks like taking screenshots and clicking.

In [50]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself.\
If the search results are vague, combine them with prior knowledge and reasonable inference to provide the best possible answer." \
""

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    # tools=[WebSearchTool(search_context_size="low")],
    tools=[duckduckgo_web_search],
    model=model,
    model_settings=ModelSettings(tool_choice="required"),
)

In [51]:
message="Latest AI Agent frameworks in 2025"

with trace("Search"):
    result=await Runner.run(search_agent,message)

display(Markdown(result.final_output))

AI agent frameworks are streamlining workflows, making them more efficient and less reliant on manual input. Popular frameworks in 2025 include LangChain, AutoGen, CrewAI, Microsoft Semantic Kernel, and the OpenAI Agents SDK. These frameworks offer tools and components that speed up the development, deployment, and management of AI systems, ranging from single-agent applications to complex multi-agent environments.

Key trends in AI agents for 2025 involve Agentic RAG, Voice Agents, AI Agent Protocols, DeepResearch Agents, and Coding Agents. Growth in AI agent-based projects has increased significantly. Frameworks provide developers with interfaces and execution platforms, functioning as operating systems within the AI technology ecosystem.


<h3>Let's try to use Structured Outputs, and include a description of the fields</h3>

In [53]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model=model,
    output_type=WebSearchPlan,
)

In [54]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='This is a direct query for AI agent frameworks specifically in the target year.', query='AI agent frameworks 2025'), WebSearchItem(reason='This helps uncover newer architectural approaches and patterns used in AI agents.', query='emerging AI agent architectures 2025'), WebSearchItem(reason='This explores the cutting edge, including novel techniques and notable projects pushing the boundaries of AI agent capabilities.', query='state-of-the-art AI agents 2025')]


In [56]:
from email.message import EmailMessage
import ssl
import smtplib

@function_tool
def send_email(subject:str, html_body:str) ->Dict[str,str]:
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = "himanshusinghvns343@gmail.com"
    msg["To"] = "himanshusinghvns343@gmail.com"
    msg.add_alternative(html_body, subtype="html")

    context = ssl.create_default_context()
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls(context=context)
        server.login(os.getenv('GMAIL_USER_ID'), os.getenv('GMAIL_API_KEY'))
        server.send_message(msg)

    return {"status": "success"}

In [58]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model=model,
)

In [64]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words." 
    "At the bottom of report provide some useful reference links to explore."\
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model=model,
    output_type=ReportData,
)

In [65]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [66]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [ ]:
query="Latest AI Agent framework in 2025"

# What are some of the most exiciting commercial applications of Autonomous Agentic AI as of July 2025

with trace("Research trace"):
    print("Starting reasearch..")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting reasearch..
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!
